In [ ]:
import neat
import gym
from gym import wrappers
import numpy as np
from multiprocessing import Pool
import pickle



np.set_printoptions(threshold=np.inf)
env = gym.make('BipedalWalker-v2')
config_path = ('./BipedalWalker.config')
config = neat.Config(neat.DefaultGenome, neat.DefaultReproduction,
                         neat.DefaultSpeciesSet, neat.DefaultStagnation,
                         config_path)

def get_fitness(genome_tuple):
    genome_id, genome = genome_tuple

    net = neat.nn.FeedForwardNetwork.create(genome, config)
    fitness_list = [0 for i in range(3)]

    for k in range(3):
        fitness = 0
        frames = 0
        observation = env.reset()
        while True:
            inputs = observation
            output = net.activate(inputs)
            output = np.clip(output, -1, 1)
            
            observation, reward, done, info = env.step(np.array(output))
            fitness += reward
            frames += 1

            if done or frames > 1000:
                if done:
                    fitness += 70
                fitness_list[k] = fitness
                break
    fitness_list = sorted(fitness_list,reverse=True)
    
    #重み付け平均
    total_fitness = (
        fitness_list[0] * 1
        + fitness_list[1] * 0.9
        + fitness_list[2] * 0.8
    ) / 2.7
    
    return total_fitness


def eval_genomes(genomes, config):
    #スレッド数
    pool = Pool(7)
    
    fitness_list = pool.map(get_fitness, genomes)
    pool.close()
    
    genomes_index = 0
    for genome_id, genome in genomes:
        genome.fitness = fitness_list[genomes_index]
        genomes_index += 1
    

def run(n):
    # Create the population, which is the top-level object for a NEAT run.
    p = neat.Population(config)

    p.add_reporter(neat.StdOutReporter(True))
    stats = neat.StatisticsReporter()
    p.add_reporter(stats)

    winner = p.run(eval_genomes, n)
    
    return p

pop = run(1500)

In [ ]:
#genome保存
genome = pop.best_genome
with open('best_genome.binaryfile', 'wb') as f:
    pickle.dump(genome, f)

In [ ]:
np.set_printoptions(threshold=np.inf)
env = gym.make('BipedalWalker-v2')
env = wrappers.Monitor(env, './mp4')
config_path = ('./BipedalWalker.config')
config = neat.Config(neat.DefaultGenome, neat.DefaultReproduction,
                         neat.DefaultSpeciesSet, neat.DefaultStagnation,
                         config_path)

with open('best_genome.binaryfile', 'rb') as f:
    genome = pickle.load(f)
net = neat.nn.FeedForwardNetwork.create(genome, config)

observation = env.reset()
frames = 0
env.render()
while 1:
    inputs = observation

    # active neurons
    output = net.activate(inputs)
    output = np.clip(output, -1, 1)
    # print(output)
    observation, reward, done, info = env.step(np.array(output))
    env.render()
    frames += 1

    if done or frames > 3000:
        break
env.close()